In [65]:
import pandas as pd

In [66]:
NOME_TABELA_SEXO = '1.3-EMP-SEXO'
NOME_TABELA_DEPARTAMENTO = '1.3-EMP-DEPTO'
NOME_TABELA_GERENTE = '1.3-EMP-GERENTE'
NOME_TABELA_UNICA = '1.3-EMP-ALL'

In [67]:
#Lendo o arquivo de dados
df = pd.read_csv('../dados.csv',dtype='object')
df.head()

,ID_EMP,NOME,SOBREN,RG,DT_NASC,ENDER,SEXO,SAL,DEPTO,ID_GER
0,17206-2,JOSE,DA SILVA,24535357,25/02/70,"Rua Froriano Peixoto,25",M,7000,1,-
1,12584-7,CECILIA,ORTIZ REZENDE,65784123,12/10/80,"Rua Marechal Deodoro da Fonseca, 335",F,3200,3,17206-2
2,16764-6,PEDRO,SILVESTRE,24789635,15/05/75,"Rua Bela Cintra,159",M,2800,4,17206-2
3,17987-5,FELIPE,GUILHERMINO,25896314,19/08/82,"Rua Bolívia, 348",M,1800,3,12584-7
4,15698-3,LUCIANA,FEITOSA,59874123,03/03/80,"Rua Presidente Dutra, 25",F,1500,4,16764-5


In [68]:
serie_empregados = df.apply(lambda x: x.to_dict(),axis=1)

In [69]:
serie_empregados.get(0)

{'ID_EMP': '17206-2',
 'NOME': 'JOSE',
 'SOBREN': 'DA SILVA',
 'RG': '24535357',
 'DT_NASC': '25/02/70',
 'ENDER': 'Rua Froriano Peixoto,25',
 'SEXO': 'M',
 'SAL': '7000',
 'DEPTO': '1',
 'ID_GER': '-'}

In [70]:
def criar_put(tabela,rowkey,familia,coluna,valor):
    return "put '"+tabela+"','"+rowkey+"','"+familia+":"+coluna+"','"+valor+"'"

In [71]:
def criar_put_tabela_unica(empregado,coluna,prefixo_familia):
    return criar_put(NOME_TABELA_UNICA,empregado['ID_EMP'],prefixo_familia+"-"+empregado[prefixo_familia],coluna,str(empregado[coluna])) 

In [72]:
def criar_by_sexo(empregado,coluna,prefixo = False):
    if (prefixo == False):
        return criar_put(NOME_TABELA_SEXO,empregado['ID_EMP'],empregado['SEXO'],coluna,str(empregado[coluna]))
    return criar_put_tabela_unica(empregado,coluna,'SEXO')

In [73]:
def criar_by_departamento(empregado,coluna,prefixo = False):
    if (prefixo == False):
        return criar_put(NOME_TABELA_DEPARTAMENTO,empregado['ID_EMP'],"D"+str(empregado['DEPTO']),coluna,str(empregado[coluna]))
    return criar_put_tabela_unica(empregado,coluna,'DEPTO')

In [91]:
def criar_by_gerente(empregado,coluna,prefixo = False):
    if (prefixo == False):
        return criar_put(NOME_TABELA_GERENTE,empregado['ID_EMP'],''+str(empregado['ID_GER']),coluna,str(empregado[coluna]))
    return criar_put_tabela_unica(empregado,coluna,'ID_GER')

In [75]:
def gerar_lista_e_colunas(nome_coluna):
    lista_unicos = df[nome_coluna].unique().tolist()
    lista_colunas = df.columns.tolist()
    lista_colunas.remove(nome_coluna)
    lista_colunas.remove('ID_EMP')
    return lista_unicos,lista_colunas

# Modelagem 1 - Três Modelos

## Modelo tabela sexo

In [80]:
def gerar_lista_sexo(prefixo=False):
    lista_sexos, lista_colunas_sexo = gerar_lista_e_colunas('SEXO')
    script_create = ""
    if (prefixo == False):
        script_create = "create '"+NOME_TABELA_SEXO+"','"+"','".join(lista_sexos)+"'"
    lista_put = []
    for empregado in serie_empregados:
        for coluna in lista_colunas_sexo:
            lista_put.append(criar_by_sexo(empregado,coluna,prefixo))
    return script_create,lista_put

## Modelo tabela departamento

In [81]:
def gerar_lista_depto(prefixo=False):
    lista_deptos,lista_colunas_deptos = gerar_lista_e_colunas('DEPTO')
    script_create = ""
    if (prefixo == False):
        script_create = "create '"+NOME_TABELA_DEPARTAMENTO+"','D"+"','D".join(lista_deptos)+"'"
    lista_put = []
    for empregado in serie_empregados:
        for coluna in lista_colunas_deptos:
            lista_put.append(criar_by_departamento(empregado,coluna,prefixo))
    return script_create, lista_put

## Modelo tabela gerente

In [82]:
def gerar_lista_gerente(prefixo=False):
    lista_gerentes,lista_colunas_gerentes = gerar_lista_e_colunas("ID_GER")
    script_create = ""
    if (prefixo == False):
        script_create = "create '"+NOME_TABELA_GERENTE+"','"+"','".join(lista_gerentes)+"'"
    lista_put = []
    for empregado in serie_empregados:
        for coluna in lista_colunas_gerentes:
            lista_put.append(criar_by_gerente(empregado,coluna,prefixo))
    return script_create,lista_put

In [83]:
script_create_by_sexo, lista_by_sexo = gerar_lista_sexo()
script_create_by_depto, lista_by_depto = gerar_lista_depto()
script_create_by_gerente, lista_by_gerente = gerar_lista_gerente()

In [84]:
lista_scripts_popular_modelo1 = [script_create_by_sexo] + lista_by_sexo
lista_scripts_popular_modelo1 += [script_create_by_depto] + lista_by_depto
lista_scripts_popular_modelo1 += [script_create_by_gerente] + lista_by_gerente


In [86]:
with open('popular_modelo1.txt', 'w') as f:
    for item in lista_scripts_popular_modelo1:
        f.write("%s\n" % item)

# Modelagem 2 - Apenas um modelo

In [101]:
lista_sexos,_ =gerar_lista_e_colunas('SEXO')
lista_deptos,_ =gerar_lista_e_colunas('DEPTO')
lista_gerentes,_ =gerar_lista_e_colunas('ID_GER')
script_tabela_unica = "create '"+NOME_TABELA_UNICA+"','SEXO-"+"','SEXO-".join(lista_sexos)+"'"
script_tabela_unica +=  ",'DEPTO-"+"','DEPTO-".join(lista_deptos)+"'"
script_tabela_unica +=  ",'ID_GER-"+"','ID_GER-".join(lista_gerentes)+"'"
script_tabela_unica

"create '1.3-EMP-ALL','SEXO-M','SEXO-F','DEPTO-1','DEPTO-3','DEPTO-4','ID_GER--','ID_GER-17206-2','ID_GER-12584-7','ID_GER-16764-5'"

In [102]:
_, lista_familia_by_sexo = gerar_lista_sexo(prefixo=True)
_, lista_familia_by_depto = gerar_lista_depto(prefixo=True)
_, lista_familia_by_gerente = gerar_lista_gerente(prefixo=True)

In [103]:
lista_scripts_popular_modelo2 = [script_tabela_unica] + lista_familia_by_sexo + lista_familia_by_depto + lista_familia_by_gerente

In [104]:
with open('popular_modelo2.txt', 'w') as f:
    for item in lista_scripts_popular_modelo2:
        f.write("%s\n" % item)